# Label The Candidate Sentences

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

from sqlalchemy import create_engine
from snorkel.labeling import PandasLFApplier
import pandas as pd

from snorkeling_helper.label_functions.disease_gene import *

In [2]:
username = "danich1"
password = "snorkel"
dbname = "pubmed_central_db"
database_str = f"postgresql+psycopg2://{username}:{password}@/{dbname}?host=/var/run/postgresql"
conn = create_engine(database_str)

In [3]:
candidates_df = pd.read_csv(
    "../annotation_conversion/output/mapped_disease_gene_candidates.tsv", 
    sep="\t"
)
candidates_df.head()

,sentence_id,document_id,text,old_candidate_id,split,Disease_cid,disease_start,disease_end,Gene_cid,gene_start,gene_end,words,lemmas,pos_tags,char_offsets,curated_dsh
0,187944955.0,23520,The decrease of sympathetic activity by the be...,21841,1,DOID:10763,331,342,1621,252,267,The|decrease|of|sympathetic|activity|by|the|be...,the|decrease|of|sympathetic|activity|by|the|be...,"DT|NN|IN|JJ|NN|IN|DT|NN|NN|,|IN|VBN|IN|DT|VBN|...",0|4|13|16|28|37|40|44|58|62|64|67|80|83|87|97|...,1.0
1,NaN,217570,Hypothalamic-pituitary-adrenal function in a w...,23356,2,DOID:2841,104,119,5443,193,197,Hypothalamic-pituitary-adrenal|function|in|a|w...,hypothalamic-pituitary-adrenal|function|in|a|w...,"JJ|NN|IN|DT|JJ|,|RB|VBN|NN|IN|CD|NNS|IN|JJ|NN|...",0|31|40|43|45|57|59|69|78|84|87|90|99|104|114|...,0.0
2,561631035.0,31141,The values for GGTP were significantly higher ...,23367,2,DOID:7148,67,68,92086,15,18,The|values|for|GGTP|were|significantly|higher|...,the|value|for|ggtp|be|significantly|higher|in|...,DT|NNS|IN|NN|VBD|RB|JJR|IN|DT|NNS|IN|NN|VBN|IN...,0|4|11|15|20|25|39|46|49|53|62|67|70|79|84|88|...,1.0
3,42895052.0,266209,"A family of antigens, referred to collectively...",23820,2,DOID:2043,154,164,3756,61,64,"A|family|of|antigens|,|referred|to|collectivel...","a|family|of|antigen|,|refer|to|collectively|as...","DT|NN|IN|NNS|,|VBN|TO|RB|IN|SYM|NN|-LRB-|NN|-R...",0|2|9|12|20|22|31|34|47|50|52|60|61|64|65|67|7...,1.0
4,275600973.0,201631,Mitochondrial ATPase activities of hepatoma BW...,24815,2,DOID:3571,35,42,514,0,19,Mitochondrial|ATPase|activities|of|hepatoma|BW...,mitochondrial|atpase|activity|of|hepatoma|bw77...,JJ|NN|NNS|IN|NN|NN|CC|NNS|NN|NNS|.,0|14|21|32|35|44|51|55|63|69|74,0.0


## Train Candidates Abstract Only

In [4]:
candidates = ",".join(map(str, candidates_df.document_id.tolist()))
sql = f'''
select * from disease_gene
where sentence_id not in ({candidates}) and 
section = 'title' or section ='abstract'
'''

candidate = pd.read_sql(sql, database_str)
print(candidate.shape)
candidate.head()

(1539670, 14)


,disease_cid,gene_cid,candidate_id,sentence_id,section,text,word,lemma,pos_tag,char_offset,disease_start,disease_end,gene_start,gene_end
0,DOID:3070,1434,121,243357468,abstract,"In conclusion, the findings demonstrated that ...","'In'|'conclusion'|','|'the'|'findings'|'demons...","'in'|'conclusion'|','|'the'|'finding'|'demonst...","'IN'|'NN'|','|'DT'|'NNS'|'VBD'|'IN'|'NNP'|'VBD...",1153|1156|1166|1168|1172|1181|1194|1199|1208|1...,12,13,21,24
1,DOID:3070,1434,122,243357468,abstract,"In conclusion, the findings demonstrated that ...","'In'|'conclusion'|','|'the'|'findings'|'demons...","'in'|'conclusion'|','|'the'|'finding'|'demonst...","'IN'|'NN'|','|'DT'|'NNS'|'VBD'|'IN'|'NNP'|'VBD...",1153|1156|1166|1168|1172|1181|1194|1199|1208|1...,21,24,42,43
2,DOID:3070,1434,124,243357460,title,Long noncoding RNA AB073614 promotes the malig...,'Long'|'noncoding'|'RNA'|'AB073614'|'promotes'...,'long'|'noncode'|'rna'|'ab073614'|'promote'|'t...,'JJ'|'VBG'|'NNP'|'NNP'|'VBZ'|'DT'|'NN'|'IN'|'N...,0|5|15|19|28|37|41|52|55|62|65|76|79|80|84|85|...,8,9,13,16
3,DOID:2355,4524,148,574284674,abstract,"As a result, the methylenetetrahydrofolate red...","'As'|'a'|'result'|','|'the'|'methylenetetrahyd...","'as'|'a'|'result'|','|'the'|'methylenetetrahyd...","'IN'|'DT'|'NN'|','|'DT'|'NN'|'NN'|'NNP'|'NN'|'...",259|262|264|270|272|276|302|312|318|327|331|33...,5,7,46,49
4,DOID:10763,173,190,277144013,abstract,The waist circumference cut points for diagnos...,'The'|'waist'|'circumference'|'cut'|'points'|'...,'the'|'waist'|'circumference'|'cut'|'point'|'f...,'DT'|'NN'|'NN'|'VBD'|'NNS'|'IN'|'VBG'|'JJ'|'NN...,754|758|764|778|782|789|793|804|814|823|824|82...,24,27,19,20


In [5]:
sen_id_map = ','.join(map(str, candidate.sentence_id.tolist()))
sql = f'''
select sentence.document_id, sentence_id 
from sentence
inner join
(
    select distinct document_id from sentence
    where section != 'title' and section !='abstract'
) as doc_ids
on doc_ids.document_id=sentence.document_id
where sentence_id in ({sen_id_map})
'''

full_text_docs_ids = pd.read_sql(sql, database_str)
print(full_text_docs_ids.shape)
full_text_docs_ids.head()

(200822, 2)


,document_id,sentence_id
0,18648522,172954
1,31323024,326031
2,28101291,406522
3,27465988,491157
4,27465988,491158


In [6]:
candidate = (
    candidate
    .merge(
        full_text_docs_ids
        .assign(full_text=True)
        [["sentence_id", "full_text"]],
        on=["sentence_id"],
        how="left"
    )
    .fillna(False)
    .assign(
        word=lambda x: x.apply(lambda row: row.word.replace("\'", "").split("|"), axis=1),
        pos_tag=lambda x: x.apply(lambda row: row.pos_tag.replace("\'", "").split("|"), axis=1)
    )
)
print(candidate.shape)
candidate.head()

(1539670, 15)


,disease_cid,gene_cid,candidate_id,sentence_id,section,text,word,lemma,pos_tag,char_offset,disease_start,disease_end,gene_start,gene_end,full_text
0,DOID:3070,1434,121,243357468,abstract,"In conclusion, the findings demonstrated that ...","[In, conclusion, ,, the, findings, demonstrate...","'in'|'conclusion'|','|'the'|'finding'|'demonst...","[IN, NN, ,, DT, NNS, VBD, IN, NNP, VBD, DT, NN...",1153|1156|1166|1168|1172|1181|1194|1199|1208|1...,12,13,21,24,True
1,DOID:3070,1434,122,243357468,abstract,"In conclusion, the findings demonstrated that ...","[In, conclusion, ,, the, findings, demonstrate...","'in'|'conclusion'|','|'the'|'finding'|'demonst...","[IN, NN, ,, DT, NNS, VBD, IN, NNP, VBD, DT, NN...",1153|1156|1166|1168|1172|1181|1194|1199|1208|1...,21,24,42,43,True
2,DOID:3070,1434,124,243357460,title,Long noncoding RNA AB073614 promotes the malig...,"[Long, noncoding, RNA, AB073614, promotes, the...",'long'|'noncode'|'rna'|'ab073614'|'promote'|'t...,"[JJ, VBG, NNP, NNP, VBZ, DT, NN, IN, NNP, IN, ...",0|5|15|19|28|37|41|52|55|62|65|76|79|80|84|85|...,8,9,13,16,True
3,DOID:2355,4524,148,574284674,abstract,"As a result, the methylenetetrahydrofolate red...","[As, a, result, ,, the, methylenetetrahydrofol...","'as'|'a'|'result'|','|'the'|'methylenetetrahyd...","[IN, DT, NN, ,, DT, NN, NN, NNP, NN, VBD, RBR,...",259|262|264|270|272|276|302|312|318|327|331|33...,5,7,46,49,False
4,DOID:10763,173,190,277144013,abstract,The waist circumference cut points for diagnos...,"[The, waist, circumference, cut, points, for, ...",'the'|'waist'|'circumference'|'cut'|'point'|'f...,"[DT, NN, NN, VBD, NNS, IN, VBG, JJ, NN, -LRB-,...",754|758|764|778|782|789|793|804|814|823|824|82...,24,27,19,20,True


In [6]:
lf_columns = (
    list(DaG['distant_supervision'].values())+
    list(DaG['text_patterns'].values())
)

lf_column_names = [col.name for col in lf_columns]

In [7]:
lf_applier = PandasLFApplier(lfs=lf_columns)
labels = lf_applier.apply(df=candidate)

100%|██████████| 1539670/1539670 [4:51:28<00:00, 88.04it/s]   


In [8]:
train_lfs_df = (
    pd.DataFrame(labels, columns=lf_column_names)
    .assign(candidate_id=candidate.candidate_id.tolist())
)
print(train_lfs_df.shape)
train_lfs_df.head()

(1539670, 25)


,LF_HETNET_DISEASES,LF_HETNET_DOAF,LF_HETNET_DisGeNET,LF_HETNET_GWAS,LF_HETNET_DaG_ABSENT,LF_DG_IS_BIOMARKER,LF_DaG_ASSOCIATION,LF_DaG_WEAK_ASSOCIATION,LF_DaG_NO_ASSOCIATION,LF_DaG_CELLULAR_ACTIVITY,...,LF_DG_PATIENT_WITH,LF_DG_CONCLUSION_TITLE,LF_DaG_NO_CONCLUSION,LF_DaG_CONCLUSION,LF_DG_DISTANCE_SHORT,LF_DG_DISTANCE_LONG,LF_DG_ALLOWED_DISTANCE,LF_DG_NO_VERB,LF_DG_CONTEXT_SWITCH,candidate_id
0,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,1,-1,-1,121
1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,1,-1,-1,122
2,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,1,-1,-1,124
3,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,1,-1,-1,148
4,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,0,-1,-1,-1,1,-1,-1,190


In [9]:
(
    train_lfs_df
    .to_csv("output/dg_abstract_train_candidates.tsv", sep="\t", index=False)
)

## Train Full Text Only

In [4]:
candidates = ",".join(map(str, candidates_df.document_id.tolist()))
sql = f'''
select * from disease_gene
where sentence_id not in ({candidates}) and 
section != 'title' and section !='abstract'
'''

candidate = pd.read_sql(sql, database_str)
print(candidate.shape)
candidate.head()

(1956984, 13)


,disease_cid,gene_cid,candidate_id,sentence_id,section,text,word,pos_tag,char_offset,disease_start,disease_end,gene_start,gene_end
0,DOID:3070,1434,97,243357535,results,"Taken together, these results indicated that A...","'Taken'|'together'|','|'these'|'results'|'indi...","'VBN'|'RB'|','|'DT'|'NNS'|'VBD'|'IN'|'NNP'|'VB...",13550|13556|13564|13566|13572|13580|13590|1359...,11,14,21,22
1,DOID:3070,1434,98,243357528,results,The findings showed a trend of inverse express...,'The'|'findings'|'showed'|'a'|'trend'|'of'|'in...,'DT'|'NNS'|'VBD'|'DT'|'NN'|'IN'|'JJ'|'NN'|'NN'...,12566|12570|12579|12586|12588|12594|12597|1260...,20,21,14,17
2,DOID:3070,1434,100,243357522,results,AB073614 positively regulated the activity of ...,'AB073614'|'positively'|'regulated'|'the'|'act...,'NNP'|'RB'|'VBD'|'DT'|'NN'|'IN'|'NNP'|'SYM'|'N...,11595|11604|11615|11625|11629|11638|11641|1164...,21,22,8,11
3,DOID:3070,1434,101,243357507,results,The data indicated that the activity of Wnt/be...,'The'|'data'|'indicated'|'that'|'the'|'activit...,'DT'|'NNS'|'VBD'|'IN'|'DT'|'NN'|'IN'|'NNP'|'SY...,7736|7740|7745|7755|7760|7764|7773|7776|7779|7...,33,36,39,40
4,DOID:3070,1434,102,243357507,results,The data indicated that the activity of Wnt/be...,'The'|'data'|'indicated'|'that'|'the'|'activit...,'DT'|'NNS'|'VBD'|'IN'|'DT'|'NN'|'IN'|'NNP'|'SY...,7736|7740|7745|7755|7760|7764|7773|7776|7779|7...,9,12,39,40


In [5]:
candidate = (
    candidate
    .assign(
        word=lambda x: x.apply(lambda row: row.word.replace("\'", "").split("|"), axis=1),
        pos_tag=lambda x: x.apply(lambda row: row.pos_tag.replace("\'", "").split("|"), axis=1)
    )
)
print(candidate.shape)
candidate.head()

(1956984, 13)


,disease_cid,gene_cid,candidate_id,sentence_id,section,text,word,pos_tag,char_offset,disease_start,disease_end,gene_start,gene_end
0,DOID:3070,1434,97,243357535,results,"Taken together, these results indicated that A...","[Taken, together, ,, these, results, indicated...","[VBN, RB, ,, DT, NNS, VBD, IN, NNP, VBD, NNP, ...",13550|13556|13564|13566|13572|13580|13590|1359...,11,14,21,22
1,DOID:3070,1434,98,243357528,results,The findings showed a trend of inverse express...,"[The, findings, showed, a, trend, of, inverse,...","[DT, NNS, VBD, DT, NN, IN, JJ, NN, NN, IN, NNP...",12566|12570|12579|12586|12588|12594|12597|1260...,20,21,14,17
2,DOID:3070,1434,100,243357522,results,AB073614 positively regulated the activity of ...,"[AB073614, positively, regulated, the, activit...","[NNP, RB, VBD, DT, NN, IN, NNP, SYM, NN, HYPH,...",11595|11604|11615|11625|11629|11638|11641|1164...,21,22,8,11
3,DOID:3070,1434,101,243357507,results,The data indicated that the activity of Wnt/be...,"[The, data, indicated, that, the, activity, of...","[DT, NNS, VBD, IN, DT, NN, IN, NNP, SYM, NN, H...",7736|7740|7745|7755|7760|7764|7773|7776|7779|7...,33,36,39,40
4,DOID:3070,1434,102,243357507,results,The data indicated that the activity of Wnt/be...,"[The, data, indicated, that, the, activity, of...","[DT, NNS, VBD, IN, DT, NN, IN, NNP, SYM, NN, H...",7736|7740|7745|7755|7760|7764|7773|7776|7779|7...,9,12,39,40


In [6]:
lf_columns = (
    list(DaG['distant_supervision'].values())+
    list(DaG['text_patterns'].values())
)

lf_column_names = [col.name for col in lf_columns]

In [7]:
lf_applier = PandasLFApplier(lfs=lf_columns)
labels = lf_applier.apply(df=candidate)

100%|██████████| 1956984/1956984 [6:39:53<00:00, 81.56it/s]    


In [8]:
train_lfs_df = (
    pd.DataFrame(labels, columns=lf_column_names)
    .assign(candidate_id=candidate.candidate_id.tolist())
)
print(train_lfs_df.shape)
train_lfs_df.head()

(1956984, 25)


,LF_HETNET_DISEASES,LF_HETNET_DOAF,LF_HETNET_DisGeNET,LF_HETNET_GWAS,LF_HETNET_DaG_ABSENT,LF_DG_IS_BIOMARKER,LF_DaG_ASSOCIATION,LF_DaG_WEAK_ASSOCIATION,LF_DaG_NO_ASSOCIATION,LF_DaG_CELLULAR_ACTIVITY,...,LF_DG_PATIENT_WITH,LF_DG_CONCLUSION_TITLE,LF_DaG_NO_CONCLUSION,LF_DaG_CONCLUSION,LF_DG_DISTANCE_SHORT,LF_DG_DISTANCE_LONG,LF_DG_ALLOWED_DISTANCE,LF_DG_NO_VERB,LF_DG_CONTEXT_SWITCH,candidate_id
0,-1,-1,-1,-1,0,1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,97
1,-1,-1,-1,-1,0,-1,1,-1,-1,-1,...,-1,-1,-1,1,0,-1,-1,-1,-1,98
2,-1,-1,-1,-1,0,1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,100
3,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,0,-1,0,-1,-1,-1,-1,101
4,-1,-1,-1,-1,0,-1,1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,102


In [9]:
(
    train_lfs_df
    .to_csv("output/dg_full_text_train_candidates.tsv", sep="\t", index=False)
)

## Dev Set

In [10]:
annotated_df = pd.read_csv(
    "../annotation_conversion/output/mapped_disease_gene_candidates.tsv", 
    sep="\t"
)
annotated_df.head()

,sentence_id,document_id,text,old_candidate_id,split,Disease_cid,disease_start,disease_end,Gene_cid,gene_start,gene_end,words,lemmas,pos_tags,char_offsets,curated_dsh
0,187944955.0,23520,The decrease of sympathetic activity by the be...,21841,1,DOID:10763,331,342,1621,252,267,The|decrease|of|sympathetic|activity|by|the|be...,the|decrease|of|sympathetic|activity|by|the|be...,"DT|NN|IN|JJ|NN|IN|DT|NN|NN|,|IN|VBN|IN|DT|VBN|...",0|4|13|16|28|37|40|44|58|62|64|67|80|83|87|97|...,1.0
1,NaN,217570,Hypothalamic-pituitary-adrenal function in a w...,23356,2,DOID:2841,104,119,5443,193,197,Hypothalamic-pituitary-adrenal|function|in|a|w...,hypothalamic-pituitary-adrenal|function|in|a|w...,"JJ|NN|IN|DT|JJ|,|RB|VBN|NN|IN|CD|NNS|IN|JJ|NN|...",0|31|40|43|45|57|59|69|78|84|87|90|99|104|114|...,0.0
2,561631035.0,31141,The values for GGTP were significantly higher ...,23367,2,DOID:7148,67,68,92086,15,18,The|values|for|GGTP|were|significantly|higher|...,the|value|for|ggtp|be|significantly|higher|in|...,DT|NNS|IN|NN|VBD|RB|JJR|IN|DT|NNS|IN|NN|VBN|IN...,0|4|11|15|20|25|39|46|49|53|62|67|70|79|84|88|...,1.0
3,42895052.0,266209,"A family of antigens, referred to collectively...",23820,2,DOID:2043,154,164,3756,61,64,"A|family|of|antigens|,|referred|to|collectivel...","a|family|of|antigen|,|refer|to|collectively|as...","DT|NN|IN|NNS|,|VBN|TO|RB|IN|SYM|NN|-LRB-|NN|-R...",0|2|9|12|20|22|31|34|47|50|52|60|61|64|65|67|7...,1.0
4,275600973.0,201631,Mitochondrial ATPase activities of hepatoma BW...,24815,2,DOID:3571,35,42,514,0,19,Mitochondrial|ATPase|activities|of|hepatoma|BW...,mitochondrial|atpase|activity|of|hepatoma|bw77...,JJ|NN|NNS|IN|NN|NN|CC|NNS|NN|NNS|.,0|14|21|32|35|44|51|55|63|69|74,0.0


In [11]:
def char_to_word(row):
    char_word_mapper = [
        (idx, int(tok)) 
        for idx, tok in enumerate(row['char_offsets'].split("|"))
    ]
    gene_end = -1
    gene_start = -1
    disease_end = -1
    disease_start = -1
    for word in char_word_mapper:
        if word[1] >= row['disease_start'] and word[1] <= row['disease_end']:
            if word[1] == row['disease_start']:
                disease_start = word[0]
            else:
                disease_end = word[0]
        if word[1] >= row['gene_start'] and word[1] <= row['gene_end']:
            if word[1] == row['gene_start']:
                gene_start = word[0]
            else:
                gene_end = word[0]

    return (
        disease_start, 
        disease_end if disease_end != -1 else disease_start+1, 
        gene_start, 
        gene_end if gene_end != -1 else gene_start+1
    )

In [12]:
fixed_map_df = (
    pd.DataFrame
    .from_dict(
        dict(
            zip(
                [
                 "disease_start", "disease_end", 
                  "gene_start","gene_end"
                ],
                zip(*
                    annotated_df
                    .apply(char_to_word, axis=1)
                    .tolist()
                )
            )
        )
    )
)
fixed_map_df

,disease_start,disease_end,gene_start,gene_end
0,52,53,40,41
1,13,14,25,26
2,11,12,3,4
3,30,31,12,13
4,4,5,0,1
...,...,...,...,...
1992,2,3,12,13
1993,17,19,4,5
1994,17,21,29,30
1995,23,24,9,10


In [13]:
fixed_annotated_df=(
    annotated_df
    .assign(
        disease_start = fixed_map_df.disease_start.tolist(),
        disease_end = fixed_map_df.disease_end.tolist(),
        gene_start = fixed_map_df.gene_start.tolist(),
        gene_end = fixed_map_df.gene_end.tolist()
    )
    .rename(index=str, columns={
        "Disease_cid":"disease_cid",
        "Gene_cid":"gene_cid",
        "words":"word",
        "lemmas":"lemma",
        "pos_tags":"pos_tag"
    })
    .assign(
        lemma=lambda x: x.lemma.apply(lambda s: s.split("|")),
        word=lambda x: x.word.apply(lambda s: s.split("|")),
        pos_tag=lambda x: x.pos_tag.apply(lambda s: s.split("|"))
    )
)
fixed_annotated_df

,sentence_id,document_id,text,old_candidate_id,split,disease_cid,disease_start,disease_end,gene_cid,gene_start,gene_end,word,lemma,pos_tag,char_offsets,curated_dsh
0,187944955.0,23520,The decrease of sympathetic activity by the be...,21841,1,DOID:10763,52,53,1621,40,41,"[The, decrease, of, sympathetic, activity, by,...","[the, decrease, of, sympathetic, activity, by,...","[DT, NN, IN, JJ, NN, IN, DT, NN, NN, ,, IN, VB...",0|4|13|16|28|37|40|44|58|62|64|67|80|83|87|97|...,1.0
1,NaN,217570,Hypothalamic-pituitary-adrenal function in a w...,23356,2,DOID:2841,13,14,5443,25,26,"[Hypothalamic-pituitary-adrenal, function, in,...","[hypothalamic-pituitary-adrenal, function, in,...","[JJ, NN, IN, DT, JJ, ,, RB, VBN, NN, IN, CD, N...",0|31|40|43|45|57|59|69|78|84|87|90|99|104|114|...,0.0
2,561631035.0,31141,The values for GGTP were significantly higher ...,23367,2,DOID:7148,11,12,92086,3,4,"[The, values, for, GGTP, were, significantly, ...","[the, value, for, ggtp, be, significantly, hig...","[DT, NNS, IN, NN, VBD, RB, JJR, IN, DT, NNS, I...",0|4|11|15|20|25|39|46|49|53|62|67|70|79|84|88|...,1.0
3,42895052.0,266209,"A family of antigens, referred to collectively...",23820,2,DOID:2043,30,31,3756,12,13,"[A, family, of, antigens, ,, referred, to, col...","[a, family, of, antigen, ,, refer, to, collect...","[DT, NN, IN, NNS, ,, VBN, TO, RB, IN, SYM, NN,...",0|2|9|12|20|22|31|34|47|50|52|60|61|64|65|67|7...,1.0
4,275600973.0,201631,Mitochondrial ATPase activities of hepatoma BW...,24815,2,DOID:3571,4,5,514,0,1,"[Mitochondrial, ATPase, activities, of, hepato...","[mitochondrial, atpase, activity, of, hepatoma...","[JJ, NN, NNS, IN, NN, NN, CC, NNS, NN, NNS, .]",0|14|21|32|35|44|51|55|63|69|74,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,NaN,28109040,"RESULTS: CLL/SLL is positive for CD5, CD19, CD...",36398561,1,DOID:2531,2,3,6693,12,13,"[RESULTS, :, CLL/SLL, is, positive, for, CD5, ...","[result, :, cll/sll, be, positive, for, cd5, ,...","[NNS, :, NN, VBZ, JJ, IN, NN, ,, NN, ,, NN, ,,...",0|7|9|17|20|29|33|36|38|42|44|48|50|55|59|64|6...,1.0
1993,168285630.0,28247028,"In uninfected tissues, occludin and ZO-1 forme...",36398726,1,DOID:1324,17,19,100506658,4,5,"[In, uninfected, tissues, ,, occludin, and, ZO...","[in, uninfected, tissue, ,, occludin, and, zo-...","[IN, JJ, NNS, ,, NN, CC, NN, VBD, JJ, NNS, IN,...",0|3|14|21|23|32|36|41|48|58|69|72|81|92|98|103...,0.0
1994,561670111.0,28137149,CONCLUSIONS: To our knowledge we report for th...,36400467,1,DOID:263,17,21,1393,29,30,"[CONCLUSIONS, :, To, our, knowledge, we, repor...","[conclusion, :, to, we, knowledge, we, report,...","[NNS, :, TO, PRP$, NN, PRP, VBP, IN, DT, JJ, N...",0|11|13|16|20|30|33|40|44|48|54|59|63|67|79|82...,1.0
1995,NaN,28197629,Upon knockdown of VPS36 an increase of RAB27A ...,36401956,1,DOID:1612,23,24,5874,9,10,"[Upon, knockdown, of, VPS36, an, increase, of,...","[upon, knockdown, of, vps36, a, increase, of, ...","[IN, NN, IN, NN, DT, NN, IN, NN, CC, NN, NN, ,...",0|5|15|18|24|27|36|39|46|50|57|67|69|73|77|82|...,1.0


In [14]:
lf_columns = (
    list(DaG['distant_supervision'].values())+
    list(DaG['text_patterns'].values())
)

lf_column_names = [col.name for col in lf_columns]

In [15]:
lf_applier = PandasLFApplier(lfs=lf_columns)
labels = lf_applier.apply(df=fixed_annotated_df)

100%|██████████| 1997/1997 [00:24<00:00, 82.92it/s] 


In [16]:
dev_lfs_df = (
    pd.DataFrame(labels, columns=lf_column_names)
    #.assign(candidate_id=fixed_annotated_df.candidate_id.tolist())
)
print(dev_lfs_df.shape)
dev_lfs_df.head()

(1997, 24)


,LF_HETNET_DISEASES,LF_HETNET_DOAF,LF_HETNET_DisGeNET,LF_HETNET_GWAS,LF_HETNET_DaG_ABSENT,LF_DG_IS_BIOMARKER,LF_DaG_ASSOCIATION,LF_DaG_WEAK_ASSOCIATION,LF_DaG_NO_ASSOCIATION,LF_DaG_CELLULAR_ACTIVITY,...,LF_DG_DIAGNOSIS,LF_DG_PATIENT_WITH,LF_DG_CONCLUSION_TITLE,LF_DaG_NO_CONCLUSION,LF_DaG_CONCLUSION,LF_DG_DISTANCE_SHORT,LF_DG_DISTANCE_LONG,LF_DG_ALLOWED_DISTANCE,LF_DG_NO_VERB,LF_DG_CONTEXT_SWITCH
0,-1,-1,-1,-1,0,1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,-1,-1
1,-1,-1,-1,-1,0,1,-1,-1,-1,-1,...,-1,1,-1,0,-1,-1,-1,1,-1,-1
2,-1,-1,-1,-1,0,1,-1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,-1,-1
3,-1,-1,-1,-1,0,-1,1,-1,-1,-1,...,-1,-1,-1,-1,1,-1,-1,1,-1,-1
4,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,-1,0,-1,0,-1,-1,0,-1


In [17]:
(
    dev_lfs_df
    .assign(
        split=fixed_annotated_df.split.tolist(),
        document_id=fixed_annotated_df.document_id.tolist(),
        curated_dsh=fixed_annotated_df.curated_dsh.tolist()
    )
    .to_csv("output/dg_dev_test_candidates.tsv", sep="\t", index=False)
)